### 채용 정보 

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
import random

# 기본 URL 및 페이지 범위 설정
base_url = "https://www.saramin.co.kr/zf_user/jobs/list/job-category"
page_range = range(1, 41)  # 1페이지부터 40페이지까지

# User-Agent 설정
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
}

# 데이터 저장할 리스트
data = []
rec_idx_set = set()  # rec_idx를 저장할 집합 (중복 제거를 위해)

# 각 페이지에 대해 요청 수행
for page in page_range:
    # 각 페이지에 대한 URL 생성
    url = f"{base_url}?page={page}&cat_kewd=2248%2C82%2C83%2C84%2C87&search_optional_item=n&search_done=y&panel_count=y&preview=y&isAjaxRequest=0&page_count=50&sort=RL&type=job-category&is_param=1&isSearchResultEmpty=1&isSectionHome=0&searchParamCount=1#searchTitle"

    # HTTP GET 요청
    response = requests.get(url, headers=headers, allow_redirects=True)

    # HTTP 응답 상태 코드 확인
    if response.status_code == 200:
        # BeautifulSoup으로 HTML 파싱
        soup = BeautifulSoup(response.text, 'html.parser')

        # 구인 정보가 담긴 div 선택
        job_list = soup.find_all('div', class_='box_item')

        # 구인 정보 추출
        for job in job_list:
            company = job.find('a', class_='str_tit')
            title = job.find('div', class_='job_tit').find('a')
            location = job.find('p', class_='work_place')
            career = job.find('p', class_='career').get_text(strip=True) if job.find('p', class_='career') else '정보 없음'
            education = job.find('p', class_='education').get_text(strip=True) if job.find('p', class_='education') else '정보 없음'
            job_sector = job.find('span', class_='job_sector')

            # job_sector 정보를 리스트로 저장
            job_sector_list = [sector.get_text(strip=True) for sector in job_sector.find_all('span')] if job_sector else []

            # career를 리스트로 변환
            career_list = [c.strip() for c in career.split('·')]  # '·'를 기준으로 분리

            # education에서 '↑' 기호를 제거
            education = education.replace('↑', '')

            # rec_idx 추출
            rec_idx = title['href'].split('rec_idx=')[-1].split('&')[0]  # rec_idx 값을 추출
            rec_idx_set.add(rec_idx)  # rec_idx를 집합에 추가하여 중복 제거

            # 데이터 딕셔너리 생성
            job_data = {
                "company": company.get_text(strip=True) if company else '정보 없음',
                "title": title.get_text(strip=True) if title else '정보 없음',
                "link": 'https://www.saramin.co.kr' + title['href'] if title else '정보 없음',
                "location": location.get_text(strip=True) if location else '정보 없음',
                "career": career_list,
                "education": education.strip() if education else '정보 없음',
                "job_sector": job_sector_list,
                "view": 0  # view 필드를 추가하고 기본값 0으로 설정
            }

            data.append(job_data)

            # 일정한 간격 두기 (1초에서 3초 사이의 랜덤 값)
            time.sleep(random.uniform(1, 3))

    else:
        print(f"Failed to retrieve data from page {page}: {response.status_code}")

# JSON 파일로 채용 공고 데이터 저장
with open('job_data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("데이터가 job_data.json 파일로 저장되었습니다.")


### 기업정보 아이디

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
import random
import pandas as pd  # pandas를 추가합니다.

# 기본 URL 및 페이지 범위 설정
base_url = "https://www.saramin.co.kr/zf_user/jobs/list/job-category"
page_range = range(1, 41)  # 1페이지부터 40페이지까지

# User-Agent 설정
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
}

# 데이터 저장할 리스트
data = []

# 각 페이지에 대해 요청 수행
for page in page_range:
    # 각 페이지에 대한 URL 생성
    url = f"{base_url}?page={page}&cat_kewd=2248%2C82%2C83%2C84%2C87&search_optional_item=n&search_done=y&panel_count=y&preview=y&isAjaxRequest=0&page_count=20&sort=RL&type=job-category&is_param=1&isSearchResultEmpty=1&isSectionHome=0&searchParamCount=1#searchTitle"

    # HTTP GET 요청
    response = requests.get(url, headers=headers, allow_redirects=True)

    # HTTP 응답 상태 코드 확인
    if response.status_code == 200:
        # BeautifulSoup으로 HTML 파싱
        soup = BeautifulSoup(response.text, 'html.parser')

        # 구인 정보가 담긴 div 선택
        job_list = soup.find_all('div', class_='box_item')

        # 구인 정보 추출
        for job in job_list:
            company = job.find('a', class_='str_tit')

            # 데이터 딕셔너리 생성
            job_data = company['href'].split('csn=')[-1] if company and company['href'] else '정보 없음'  # csn 값만 저장

            data.append(job_data)

            # 일정한 간격 두기 (1초에서 3초 사이의 랜덤 값)
            time.sleep(random.uniform(1, 3))

    else:
        print(f"Failed to retrieve data from page {page}: {response.status_code}")

data = list(set(data))

# JSON 파일로 저장
with open('company_id.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("데이터가 JSON 파일과 CSV 파일로 저장되었습니다.")


### 기업정보가 등록되지 않은 요소는 삭제하기

In [ ]:
import json

# JSON 파일 경로
file_path = 'company_id.json'

# JSON 파일 읽기
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)  # JSON 데이터 파싱하여 data 리스트에 저장

# '/'로 시작하는 요소를 제거
data = [item for item in data if not item.startswith('/')]

# 결과 출력
print("데이터에서 '/'로 시작하는 요소가 삭제되었습니다.")
print(data)  # 최종 필터링된 결과 출력


### 필요한 정보만 가져오도록 

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
import random
import re  # 정규 표현식 사용을 위해 import

# User-Agent 설정
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
}

# 데이터 저장할 리스트
data_list = []

# 각 페이지에 대해 요청 수행
for link in data:  # data 리스트에 csn 링크가 들어있다고 가정
    # 각 페이지에 대한 URL 생성
    url = f"https://www.saramin.co.kr/zf_user/company-info/view?csn={link}"
    
    # HTTP GET 요청
    response = requests.get(url, headers=headers, allow_redirects=True)

    # HTTP 응답 상태 코드 확인
    if response.status_code == 200:
        # BeautifulSoup으로 HTML 파싱
        soup = BeautifulSoup(response.text, 'html.parser')

        # 회사 정보 추출
        company_info = {
            "company_name": "정보 없음",
            "company_type": "정보 없음",
            "employee_count": 0,
            "industry": "정보 없음",
            "ceo_name": "정보 없음",
            "website": "정보 없음",
            "description": "정보 없음",
            "address": "정보 없음",
            "established_date": "정보 없음",
            "revenue": "정보 없음",  # revenue 초기값을 "정보 없음"으로 설정
            "salary": "정보 없음"  # salary 초기값을 "정보 없음"으로 설정
        }

        # 회사명 추출 및 "기업정보" 제거
        company_name_elem = soup.find('h1', class_='tit_company')
        if company_name_elem:
            company_info['company_name'] = company_name_elem.get_text(strip=True).replace('기업정보', '').strip()

        # 업력, 기업형태, 사원수 추출
        company_summary_items = soup.find_all('li', class_='company_summary_item')
        for item in company_summary_items:
            title = item.find('strong', class_='company_summary_tit').get_text(strip=True)
            description = item.find('p', class_='company_summary_desc').get_text(strip=True)

            # 설립일 정보에서 '설립' 단어 제거
            if '설립' in description:
                establishment_date = re.sub("설립", "", description).strip()  # '설립' 단어 제거
                company_info['established_date'] = establishment_date

            # 사원수 정보에서 숫자만 추출하고 '명' 제거
            if '사원수' in description:
                number = re.findall(r'\d+', title)  # 숫자 추출
                company_info['employee_count'] = int(number[0]) if number else 0  # '명' 제거 및 숫자형으로 변환

            # 기업형태 정보
            elif '기업형태' in description:
                company_info['company_type'] = title if title else '정보 없음'

        # 매출액 추출
        revenue_elem = soup.find('strong', class_='company_summary_tit', text=re.compile(r'억|만원'))
        if revenue_elem:
            company_info['revenue'] = revenue_elem.get_text(strip=True)  # 매출액 정보 그대로 저장
        else:
            company_info['revenue'] = "정보 없음"  # 매출액 정보가 없으면 "정보 없음"으로 설정

        # 연봉 정보 추출
        salary_url = f"https://www.saramin.co.kr/zf_user/company-info/view-inner-salary?csn={link}"
        salary_response = requests.get(salary_url, headers=headers)  # 연봉 정보를 위한 추가 요청
        if salary_response.status_code == 200:
            salary_soup = BeautifulSoup(salary_response.text, 'html.parser')
            salary_elem = salary_soup.find('p', class_='average_currency')
            if salary_elem:
                salary_value = salary_elem.find('em')
                if salary_value:
                    company_info['salary'] = salary_value.get_text(strip=True)  # 연봉 정보를 그대로 저장
                else:
                    company_info['salary'] = "정보 없음"  # 연봉 정보가 없으면 "정보 없음"으로 설정
            else:
                company_info['salary'] = "정보 없음"  # 연봉 정보 요청이 성공했으나 요소가 없을 경우
        else:
            company_info['salary'] = "정보 없음"  # 연봉 정보 요청 실패 시 "정보 없음"

        # 업종, 대표자명, 홈페이지, 사업내용, 주소 추출
        details = soup.find_all('div', class_='company_details_group')
        for detail in details:
            title = detail.find('dt', class_='tit').get_text(strip=True)
            description = detail.find('dd', class_='desc').get_text(strip=True)
            if title == '업종':
                company_info['industry'] = description.strip()  # 업종 정보 저장
            elif title == '대표자명':
                company_info['ceo_name'] = description.strip()  # CEO 이름 저장
            elif title == '홈페이지':
                company_info['website'] = description.strip()  # 웹사이트 저장
            elif title == '사업내용':
                company_info['description'] = description.strip()  # 설명 저장
            elif title == '주소':
                company_info['address'] = description.strip()  # 주소 저장

        # 회사 정보 딕셔너리를 리스트에 추가
        data_list.append(company_info)

        # 일정한 간격 두기 (1초에서 3초 사이의 랜덤 값)
        time.sleep(random.uniform(1, 3))

    else:
        print(f"Failed to retrieve data from {url}: {response.status_code}")

# JSON 파일로 저장
with open('company_info.json', 'w', encoding='utf-8') as f:
    json.dump(data_list, f, ensure_ascii=False, indent=4)

print("데이터가 JSON 파일로 저장되었습니다.")


In [ ]:
import json

# JSON 파일 경로
file_path = 'company_info.json'

# 문자열 형태의 revenue를 숫자로 변환하는 함수
def convert_revenue_to_number(revenue):
    if revenue == "정보 없음":
        return 0  # "정보 없음"일 경우 0으로 설정

    total = 0
    # 공백을 기준으로 분리
    parts = revenue.split(" ")

    # 각 요소에 대해 처리
    for part in parts:
        # 숫자와 단위를 분리
        num_str = part[:-1]  # 마지막 문자(단위)를 제외한 부분
        unit = part[-1]  # 기본적으로 마지막 문자(단위)

        # 마지막 두 문자가 "만원"인 경우 처리
        if len(part) > 2 and part[-2:] == "만원":
            unit = "만원"  # 단위를 "만원"으로 설정
            num_str = part[:-2]  # "만원" 부분 제거
        else:
            unit = part[-1]  # 단위가 "억"이나 "조"일 경우

        # 숫자에서 ','를 제거
        num_str = num_str.replace(",", "")

        # 각 단위에 따라 계산
        if "조" in unit:
            total += int(num_str) * 1_000_000_000_000  # 조 단위
        elif "억" in unit:
            total += int(num_str) * 1_000_000_000  # 억 단위
        elif "만원" in unit:
            total += int(num_str) * 10_000  # 만원 단위

    return total

# JSON 파일 읽기
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)  # JSON 데이터 파싱

# revenue 필드 변환
for company in data:
    # 기존 revenue 값을 revenue2로 저장
    company['revenue2'] = company['revenue']
    # revenue 값을 숫자로 변환
    company['revenue'] = convert_revenue_to_number(company['revenue'])  # revenue 변환

# 수정된 데이터를 JSON 파일에 저장
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("revenue 필드가 변환되고 기존 값은 revenue2로 저장되었습니다.")


In [ ]:
import json

# JSON 파일 경로
job_data_file = 'job_data.json'
company_info_file = 'company_info.json'

# job_data.json 파일 읽기
with open(job_data_file, 'r', encoding='utf-8') as f:
    job_data = json.load(f)  # JSON 데이터 파싱

# company_info.json 파일 읽기
with open(company_info_file, 'r', encoding='utf-8') as f:
    company_info_data = json.load(f)  # JSON 데이터 파싱

# 회사명을 키로 급여 정보를 매핑
salary_mapping = {company['company_name']: company['salary'] for company in company_info_data}

# 급여 정보를 숫자형으로 변환하는 함수
def convert_salary_to_number(salary):
    if salary == "정보 없음":
        return 0  # 급여 정보가 없으면 0으로 설정
    try:
        # 급여 문자열에서 숫자만 추출하고 변환
        salary_number = int(salary.replace(",", "").replace("만원", "").strip()) * 10000
        return salary_number
    except ValueError:
        return 0  # 변환이 실패할 경우 0으로 설정

# job_data에 급여 정보 추가
for job in job_data:
    company_name = job.get('company')
    salary_info = salary_mapping.get(company_name, "정보 없음")  # 급여 정보 가져오기
    job['salary'] = convert_salary_to_number(salary_info)  # 숫자형으로 변환하여 추가

# 수정된 데이터를 job_data.json 파일에 저장
with open(job_data_file, 'w', encoding='utf-8') as f:
    json.dump(job_data, f, ensure_ascii=False, indent=4)

print("급여 정보가 숫자형으로 job_data.json에 추가되었습니다.")
